## Load Packages

In [1]:
# Load packages
import os
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import pickle
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

## Define Function for Running SQL Files

In [2]:
# Get SQL query file as data input 
def get_sql_file_as_text(file_path):
  
  with open(file_path, 'r') as f:
    sql_content = f.read()
  return sql_content

## Import Table from XLSX and Add Into Snowflake as TABLE

In [3]:
# Get contact owners sheet as table
# Read contact_owners.xlsx into pandas table
df = pd.read_excel('contact_owners.xlsx')
df.head()

,Record ID,CardUp Contact Owner Current Value,CardUp Contact Owner Change Date (Asia/Singapore),CardUp Contact Owner Previous Value (1),CardUp Contact Owner Change Date (1) (Asia/Singapore),CardUp Contact Owner Previous Value (2),CardUp Contact Owner Change Date (2) (Asia/Singapore),...,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,56560804227,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
1,56559992854,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
2,56558354989,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
3,56559482221,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
4,56490172812,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT


In [4]:
# Rename Columns

df = df.rename(columns = {
    'Record ID': 'RECORD_ID',
    'CardUp Contact Owner Current Value': 'CURRENT_CONTACT_OWNER',
    'CardUp Contact Owner Change Date (Asia/Singapore)': 'CURRENT_CHANGE_DATE',
    'CardUp Contact Owner Previous Value (1)': 'PREV_CONTACT_OWNER_1',
    'CardUp Contact Owner Change Date (1) (Asia/Singapore)': 'PREV_CHANGE_DATE_1',
    'CardUp Contact Owner Previous Value (2)': 'PREV_CONTACT_OWNER_2',
    'CardUp Contact Owner Change Date (2) (Asia/Singapore)': 'PREV_CHANGE_DATE_2',
    '...': 'PREV_CONTACT_OWNER_3',
    'Unnamed: 8': 'PREV_CHANGE_DATE_3',
    'Unnamed: 9': 'PREV_CONTACT_OWNER_4',
    'Unnamed: 10': 'PREV_CHANGE_DATE_4',
})

df.head()

,RECORD_ID,CURRENT_CONTACT_OWNER,CURRENT_CHANGE_DATE,PREV_CONTACT_OWNER_1,PREV_CHANGE_DATE_1,PREV_CONTACT_OWNER_2,PREV_CHANGE_DATE_2,PREV_CONTACT_OWNER_3,PREV_CHANGE_DATE_3,PREV_CONTACT_OWNER_4,PREV_CHANGE_DATE_4
0,56560804227,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
1,56559992854,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
2,56558354989,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
3,56559482221,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
4,56490172812,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT


In [5]:
# Remove Rows with Missing Values in 'CURRENT_CONTACT_OWNER' Column
df.dropna(subset=['CURRENT_CONTACT_OWNER'], inplace=True)
len(df)

17351

In [6]:
df.head()

,RECORD_ID,CURRENT_CONTACT_OWNER,CURRENT_CHANGE_DATE,PREV_CONTACT_OWNER_1,PREV_CHANGE_DATE_1,PREV_CONTACT_OWNER_2,PREV_CHANGE_DATE_2,PREV_CONTACT_OWNER_3,PREV_CHANGE_DATE_3,PREV_CONTACT_OWNER_4,PREV_CHANGE_DATE_4
5,56498899716,Juliani Sarah (Deactivated User),2024-10-08 02:43:00,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
13,56577450520,Leia Ng,2024-09-25 15:59:00,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
20,56498377789,Juliani Sarah (Deactivated User),2024-10-08 02:43:00,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
24,56550115092,Terence Chua,2024-09-25 15:58:00,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
25,56527492928,Ashley Woo,2024-09-25 15:58:00,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT


In [7]:
# Change date format for exporting to SQL
import datetime

df['CURRENT_CHANGE_DATE'] = df['CURRENT_CHANGE_DATE'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['PREV_CHANGE_DATE_1'] = df['PREV_CHANGE_DATE_1'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['PREV_CHANGE_DATE_2'] = df['PREV_CHANGE_DATE_2'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['PREV_CHANGE_DATE_3'] = df['PREV_CHANGE_DATE_3'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['PREV_CHANGE_DATE_4'] = df['PREV_CHANGE_DATE_4'].dt.strftime('%Y-%m-%d %H:%M:%S')

df.head()

,RECORD_ID,CURRENT_CONTACT_OWNER,CURRENT_CHANGE_DATE,PREV_CONTACT_OWNER_1,PREV_CHANGE_DATE_1,PREV_CONTACT_OWNER_2,PREV_CHANGE_DATE_2,PREV_CONTACT_OWNER_3,PREV_CHANGE_DATE_3,PREV_CONTACT_OWNER_4,PREV_CHANGE_DATE_4
5,56498899716,Juliani Sarah (Deactivated User),2024-10-08 02:43:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,56577450520,Leia Ng,2024-09-25 15:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,56498377789,Juliani Sarah (Deactivated User),2024-10-08 02:43:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,56550115092,Terence Chua,2024-09-25 15:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,56527492928,Ashley Woo,2024-09-25 15:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Establish connection with Snowflake
conn = snowflake.connector.connect(connection_name = "fundingsociety.sg.ap-southeast-1.aws.privatelink")
cursor = conn.cursor()

In [9]:
# Export to SQL
cursor.execute("""
CREATE OR REPLACE TABLE DEV.SBOX_SHILTON.CONTACT_OWNERS_HISTORICAL
               (
    RECORD_ID INT,
    CURRENT_CONTACT_OWNER VARCHAR(255),
    CURRENT_CHANGE_DATE TIMESTAMP,
    PREV_CONTACT_OWNER_1 VARCHAR(255),
    PREV_CHANGE_DATE_1 TIMESTAMP,
    PREV_CONTACT_OWNER_2 VARCHAR(255),
    PREV_CHANGE_DATE_2 TIMESTAMP,
    PREV_CONTACT_OWNER_3 VARCHAR(255),
    PREV_CHANGE_DATE_3 TIMESTAMP,
    PREV_CONTACT_OWNER_4 VARCHAR(255),
    PREV_CHANGE_DATE_4 TIMESTAMP
);
""")

# Commit the changes
conn.commit()

write_pandas(
    conn,
    df,
    table_name='CONTACT_OWNERS_HISTORICAL',
    database='DEV',
    schema='SBOX_SHILTON',
    overwrite=True,
    auto_create_table=False)

/var/folders/sp/wnd46psj0r5_ys790_zv7xgm0000gr/T/ipykernel_84524/3374967899.py:22: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  write_pandas(


(True,
 1,
 17351,
 [('xutdtbwmiq/file0.txt',
   'LOADED',
   17351,
   17351,
   1,
   0,
   None,
   None,
   None,
   None)])

## Process Table

### Unpivot Table

In [10]:
# Get SQL text file
sql_file = "01 Unpivot Contact Owners Table.sql"
sql_text = get_sql_file_as_text(sql_file)

# Execute Query
cursor.execute(sql_text)


In [11]:
# Get Sample Data

testdata = conn.cursor().execute('''
select * from DEV.SBOX_SHILTON.CONTACT_OWNERS_HISTORICAL_UNPIVOT                                
''').fetch_pandas_all()

testdata.head(10)

,RECORD_ID,CONTACT_OWNER_VALUE,CONTACT_OWNER_DATE,ORDER_OWNER
0,56580328842,Ashley Woo,2024-10-08 02:43:00,1
1,56558162015,Juliani Sarah (Deactivated User),2024-10-08 02:43:00,1
2,56558684437,Juliani Sarah (Deactivated User),2024-10-08 02:43:00,1
3,56560579446,Keegan Loh,2024-09-25 15:59:00,1
4,56558935981,Juliani Sarah (Deactivated User),2024-10-08 02:43:00,1
5,56500113251,Terence Chua,2024-09-25 15:58:00,1
6,75129203304,Keegan Loh,2024-11-05 15:24:00,1
7,56549505918,Ashley Woo,2024-09-25 15:58:00,1
8,56576758676,Juliani Sarah (Deactivated User),2024-10-08 02:43:00,1
9,56582364000,Keegan Loh,2024-09-25 16:24:00,1


## Categorize Inbound, Outbound, Self-Serve, and Legacy

Inbound: Contact goes to Ashley first, and then assigned to Salesperson <br>
Outbound: Contact goes to Salesperson first <br>
Self-serve: Contact goes to Ashley and not assigned to Salesperson <br>
Legacy: Everyone else (mostly those handled by legacy salespeople)

In [12]:
# Get SQL text file
sql_file = "02 Inbound Outbound.sql"
sql_text = get_sql_file_as_text(sql_file)

# Execute Query
cursor.execute(sql_text)


In [13]:
# Get Sample Data

testdata = conn.cursor().execute('''
select * from DEV.SBOX_SHILTON.CONTACT_OWNERS_HISTORICAL_UNPIVOT_INBOUND_OUTBOUND                                
''').fetch_pandas_all()

testdata.head(10)

,RECORD_ID,OUTBOUND_INBOUND_LEAD
0,56577450520,Outbound
1,56558345010,Legacy
2,56549521292,Outbound
3,56558596927,Legacy
4,56574533781,Legacy
5,56501959092,Outbound
6,56508203117,Legacy
7,56561404962,Outbound
8,56549521294,Outbound
9,56558323281,Legacy


## Create F30D Tx Post-onboarding Value Table

In [14]:
# Get SQL text file
sql_file = "03 F30D Tx Value After Onboarding.sql"
sql_text = get_sql_file_as_text(sql_file)

# Execute Query
cursor.execute(sql_text)


In [15]:
# Get Sample Data

testdata = conn.cursor().execute('''
select * from DEV.SBOX_SHILTON.CARDUP_B2B_SG_USERID_F30DMAKE_F30DCOLLECT_AMOUNT                                
''').fetch_pandas_all()

testdata.head(10)

,COMPANY_ID,CARDUP_PAYMENT_PAYMENT_TYPE_MAKE,CARDUP_PAYMENT_PAYMENT_TYPE_COLLECT,FIRST_30D_PAYMENT_AMOUNT_MAKE,FIRST_30D_PAYMENT_AMOUNT_COLLECT
0,1991,Supplier,None,28253.113318870000,None
1,1986,Supplier,None,123558.361858140000,None
2,2114,Supplier,None,34734.391393020000,None
3,892,GST,None,14879.694922400000,None
4,4405,Rent,None,308.368172200000,None
5,1268,Rent,None,11353.487750000000,None
6,3255,Rent,None,5807.029800000000,None
7,3174,Supplier,None,6578.143798160000,None
8,3639,Rent,None,6402.622600000000,None
9,1254,Rent,None,2931.507761600000,None


## Main Table for Analysis

In [16]:
# Get SQL text file
sql_file = "04 Main Table For Salespeople Perf Analysis.sql"
sql_text = get_sql_file_as_text(sql_file)

# Execute Query
cursor.execute(sql_text)

In [17]:
# Get Sample Data

testdata = conn.cursor().execute('''
select * from DEV.SBOX_SHILTON.CARDUP_B2B_SG_FUNNEL_WITH_HISTORICAL_TAGGING
''').fetch_pandas_all()

testdata.head(10)

,RECORD_ID,COMPANY_ID,USER_ID,CONTACT_OWNER,CONTACT_OWNER_HISTORICAL,HS_CREATE_DATE,SIGN_UP_DATE,IIC_DATE,INDUSTRY,OUTBOUND_INBOUND_LEAD,...,FIRST_PAYMENT_DATE_MAKE,FIRST_PAYMENT_DATE_COLLECT,FIRST_30D_PAYMENT_AMOUNT_MAKE,FIRST_30D_PAYMENT_AMOUNT_COLLECT,DAYS_LEAD_TO_SIGN_UP,DAYS_SIGN_UP_TO_ONBOARDED_MAKE,DAYS_ONBOARDED_MAKE_TO_FIRST_PAYMENT_MAKE,DAYS_IIC_TO_CSS,DAYS_CSS_TO_ONBOARDED_COLLECT,DAYS_ONBOARDED_COLLECT_TO_FIRST_PAYMENT_COLLECT
0,56552236959,NaN,9698,None,None,2018-09-10 13:00:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,56510926764,NaN,114421,None,None,2023-04-12 09:39:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,56500098992,NaN,107743,None,None,2023-03-15 14:26:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,62337041160,NaN,231067,None,None,2024-09-12 09:01:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,56549616311,NaN,18649,None,None,2019-06-03 14:51:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,56579178768,NaN,119868,None,None,2023-05-08 03:16:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,56580342340,NaN,192526,None,None,2024-04-21 09:10:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,56577155421,NaN,123606,None,None,2023-05-29 02:21:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,56550202725,NaN,53608,None,None,2021-07-11 15:19:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,56526466114,NaN,14410,None,None,2019-01-11 16:00:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## For Report

In [18]:
overall_funnel = conn.cursor().execute('''
select * from DEV.SBOX_SHILTON.CARDUP_B2B_SG_FUNNEL_WITH_HISTORICAL_TAGGING
''').fetch_pandas_all()

overall_funnel.head(10)

,RECORD_ID,COMPANY_ID,USER_ID,CONTACT_OWNER,CONTACT_OWNER_HISTORICAL,HS_CREATE_DATE,SIGN_UP_DATE,IIC_DATE,INDUSTRY,OUTBOUND_INBOUND_LEAD,...,FIRST_PAYMENT_DATE_MAKE,FIRST_PAYMENT_DATE_COLLECT,FIRST_30D_PAYMENT_AMOUNT_MAKE,FIRST_30D_PAYMENT_AMOUNT_COLLECT,DAYS_LEAD_TO_SIGN_UP,DAYS_SIGN_UP_TO_ONBOARDED_MAKE,DAYS_ONBOARDED_MAKE_TO_FIRST_PAYMENT_MAKE,DAYS_IIC_TO_CSS,DAYS_CSS_TO_ONBOARDED_COLLECT,DAYS_ONBOARDED_COLLECT_TO_FIRST_PAYMENT_COLLECT
0,56552236959,NaN,9698,None,None,2018-09-10 13:00:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,56510926764,NaN,114421,None,None,2023-04-12 09:39:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,56500098992,NaN,107743,None,None,2023-03-15 14:26:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,62337041160,NaN,231067,None,None,2024-09-12 09:01:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,56549616311,NaN,18649,None,None,2019-06-03 14:51:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,56579178768,NaN,119868,None,None,2023-05-08 03:16:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,56580342340,NaN,192526,None,None,2024-04-21 09:10:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,56577155421,NaN,123606,None,None,2023-05-29 02:21:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,56550202725,NaN,53608,None,None,2021-07-11 15:19:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,56526466114,NaN,14410,None,None,2019-01-11 16:00:00,NaT,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [22]:
# Change HS_CREATE_DATE to datetime
overall_funnel['HS_CREATE_DATE'] = pd.to_datetime(overall_funnel['HS_CREATE_DATE'])

In [35]:
# Check for unique salespeople names
unique_contact_owner = filtered_overall_funnel['CONTACT_OWNER'].unique()
unique_contact_owner_historical = filtered_overall_funnel['CONTACT_OWNER_HISTORICAL'].unique()

print("Unique values of CONTACT_OWNER:", unique_contact_owner)
print("Unique values of CONTACT_OWNER_HISTORICAL:", unique_contact_owner_historical)

Unique values of CONTACT_OWNER: ['Jonathan Ng' 'Keegan Loh' 'Ashley Woo' 'Leia Ng' 'Terence Chua'
 'Xavier Tang']
Unique values of CONTACT_OWNER_HISTORICAL: ['   ' 'Jonathan Ng   ' 'Ashley Woo   ' 'Keegan Loh   '
 'Keegan Loh Ashley Woo  ' 'Terence Chua Keegan Loh  '
 'Xavier Tang Jonathan Ng  ' 'Xavier Tang Keegan Loh  ']


In [36]:
# Filter by dates
filtered_overall_funnel = overall_funnel[(overall_funnel['HS_CREATE_DATE'] >= '2023-11-01') & (overall_funnel['HS_CREATE_DATE'] <= '2024-10-31')]
filtered_overall_funnel.shape

# Filter by salespeople
filtered_overall_funnel = filtered_overall_funnel[
    filtered_overall_funnel['CONTACT_OWNER'].str.contains('Jonathan|Keegan|Leia', na=False) |
    filtered_overall_funnel['CONTACT_OWNER_HISTORICAL'].str.contains('Jonathan|Keegan|Leia', na=False)
]

In [40]:
# Check numbers
filtered_overall_funnel.shape

(5613, 25)

filtered_overall_funnel

HS_CREATE_DATE, SIGN_UP_DATE, MAKE_ONBOARDED_DATE, FIRST_PAYMENT_DATE_MAKE

In [41]:
# Get funnel numbers
overall_pivot_table = filtered_overall_funnel.notnull().agg('sum').to_frame('non_null_count').T

# Access counts directly from the pivot table
hs_create_date_count = overall_pivot_table['HS_CREATE_DATE']['non_null_count']
sign_up_date_count = overall_pivot_table['SIGN_UP_DATE']['non_null_count']
make_onboarded_date_count = overall_pivot_table['MAKE_ONBOARDED_DATE']['non_null_count']
first_payment_date_make_count = overall_pivot_table['FIRST_PAYMENT_DATE_MAKE']['non_null_count']

print(hs_create_date_count, sign_up_date_count, make_onboarded_date_count, first_payment_date_make_count) 

5613 351 303 255


In [47]:
# Get the median of the sum of the specified columns
median_sum = filtered_overall_funnel[['DAYS_LEAD_TO_SIGN_UP', 'DAYS_SIGN_UP_TO_ONBOARDED_MAKE', 'DAYS_ONBOARDED_MAKE_TO_FIRST_PAYMENT_MAKE']].sum(axis=1).median()
median_sum

0.0